In [6]:
from atproto import Client, client_utils
from dotenv import load_dotenv
import sqlite3
import time 
import os

load_dotenv()

username = os.getenv('username')
password = os.getenv('password')

client = Client()
client.login(username, password)

ProfileViewDetailed(did='did:plc:oc7nomnf7bsgmraom72ts3fp', handle='alex-paul-kelly.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:oc7nomnf7bsgmraom72ts3fp/bafkreiasgk56xkl76blphf7uo2yprnv45smrs56w4vouxphmaav6jibpzm@jpeg', banner=None, created_at='2024-11-19T13:11:41.438Z', description=None, display_name='', followers_count=11, follows_count=303, indexed_at='2024-11-19T13:11:41.438Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=5, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=KnownFollowers(count=5, followers=[ProfileViewBasic(did='did:plc:5hkogxu5u4wkelj6qm6iarip', handle='aisupremacy.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:5hkogxu5u4wkelj6qm6iarip/bafkreicianwa7fzkz5dcgnb344

In [7]:
# Set up SQLite database
conn = sqlite3.connect('bsky_posts.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        author TEXT,
        text TEXT
    )
''')
conn.commit()

print("Database setup complete. Starting feed...")

Database setup complete. Starting feed...


In [8]:
# Keep track of the most recent post we've seen
last_seen_post = None

while True:
    try:
        # Get the timeline
        timeline = client.get_timeline(limit=50)  # Adjust limit as needed

        # Iterate through posts, newest first
        for post in reversed(timeline.feed):
            # If we've seen this post before, skip to the next one
            if last_seen_post and post.post.cid == last_seen_post:
                break
            
            # Insert post into database
            cursor.execute('''
                INSERT INTO posts (author, text) VALUES (?, ?)
            ''', (post.post.author.handle, post.post.record.text))
            
            # # Print post details
            # print(f"Author: {post.post.author.handle}")
            # print(f"Text: {post.post.record.text}")
            # print("---")

        # Commit the changes to the database
        conn.commit()

        # Update the last seen post
        if timeline.feed:
            last_seen_post = timeline.feed[0].post.cid

        # Wait for a bit before checking again
        time.sleep(60)  # Wait for 60 seconds
        
    except KeyboardInterrupt:
        print("\nExiting the feed loop...")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Waiting before trying again...")
        time.sleep(300)  # Wait for 5 minutes before trying again

# Close the database connection
conn.close()
print("Database connection closed. Exiting program.")